In [1]:
import random
import time

## Compressed Sparse Row Representation

Compression of a matrix is a way to store a matrix in a more efficient way. In this assignment, we will look at a way to compress a matrix that has a lot of zeros in it. This is called the Compressed Sparse Row Representation of a matrix.

Let a matrix $\mathbf A$. We denote the sparse matrix as a tuple $(\mathbf{rows},\mathbf{cols}, \mathbf{data})$. The $\mathbf{cols}$ store the index of the non-zero elements in each row and $\mathbf{data}$ stores the corresponding value to it. The $i^{\text{th}}$ element of the vector $\mathbf{rows}$ stores the index where the $i^{\text{th}}$ row starts in $\mathbf{cols}$. Explaination with the help of an example will make the concept more clear.

Consider a matrix $\mathbf A \in \mathbb R^{4\times 3}$ with the following entries

$$
\mathbf A = \begin{bmatrix}
1 & 2 & 0\\
0 & 0 & 0\\
0 & 0 & 3\\
0 & 4 & 0\\
\end{bmatrix}
$$

Then, the sparse representation of $A$ is given by 

$$
\begin{align*}
\mathbf{data} &= [1,2,3,4] \\ 
\mathbf{rows} &= [0,2,2,3,4] \\ 
\mathbf{cols} &= [0,1,2,1] \\ 
\end{align*}
$$

The first element of $\mathbf{rows}$ is always $0$. The second element becomes $2$ because there are $2$ non-zero elements in the $0^\text{th}$ row. This means that, from the index $0$ to $2$ ($2$ exclusive), the vector $\mathbf{cols}$ stores the indexes of the non-zero values of the $0^\text{th}$ row. 

The third element is also $2$ because there are no non-zero element in the $1^\text{st}$ row. The fourth element is $3$ because there is only $1$ non-zero element in the $2^\text{nd}$ row. And so on. The last element is added in $\mathbf {rows}$ which stores the number of rows in $\mathbf A$. This is done so that we can easily find the number of rows in $\mathbf A$.

So all in all, apart from the first element being $0$ in the rows vectors, every other element is equal to the previous element plus the number of rows in ```index-1``` row.

In [2]:
class CSRMatrixRepresentation(object):
    """
    A class to represent a matrix in CSR format.
    """
    def __init__(self, matrix):
        """
        Initializes the CSRMatrixRepresentation class.
        
        Parameters
        ----------
        matrix : list of lists
            The matrix to be converted to CSR format.

        Initializes
        -----------
        data : list
            The non-zero elements of the matrix.
        cols : list
            The column index corresponding to the value
        rows : list
            The index of the first element in each row.
        """
        # Convert matrix to CSR format
        self.data, self.cols, self.rows = self.matrixToCSR(matrix)

    def matrixToCSR(self, matrix):
        """
        Converts a matrix to CSR format.
        
        Parameters
        ----------
        matrix : list of lists
            The matrix to be converted to CSR format.
            
        Returns
        -------
        csrMatrix : dictionary of lists
            The matrix in CSR format.
        """
        rows = [0]
        cols = []
        data = []
        
        for i in range(len(matrix)):
            for j in range(len(matrix[i])):
                if matrix[i][j] != 0:
                    data.append(matrix[i][j]) # Add non-zero element to data
                    cols.append(j) # Add column index to cols
            rows.append(len(data)) # Add index of first element in next row to rows
        return data, cols, rows

    def __str__(self):
        """
        Prints the matrix in CSR format.
        """
        return "data: " + str(self.data) + "\ncols: " + str(self.cols) + "\nrows: " + str(self.rows)

    def matrixVectorMultiply(self, vector):
        """
        Multiplies a matrix in CSR format by a vector.
        
        Parameters
        ----------
        vector : list
            The vector to be multiplied by the matrix.
            
        Returns
        -------
        result : list
            The result of the matrix-vector multiplication.
        """
        result = [] 
        for i in range(len(self.rows) - 1):
            summation = 0 
            # Iterate through non-zero elements in row
            # rows[i] is the index of the first element in row i
            # rows[i + 1] is the index of the first element in row i + 1
            for j in range(self.rows[i], self.rows[i + 1]):
                summation += self.data[j] * vector[self.cols[j]] # Multiply non-zero element by corresponding vector element
            result.append(summation)
        return result

In [3]:
# Generating a Random Sparse Matrix
def generateRandomSparseMatrix(size, density, elements_range=(-5, 5)):
    """
    Generates a random sparse matrix.
    
    Parameters
    ----------
    size : tuple 
        The size of the matrix.
    elements_range : tuple
        The range of the elements in the matrix.
    density : float
        The density of sparseness of the matrix.
        
    Returns
    -------
    matrix : list of lists
        The random sparse matrix.
    """
    matrix = []
    for i in range(size[0]):
        row = []
        for j in range(size[1]):
            # random.random() generates a number between 0 and 1 uniformly
            # if random.random() < density, then the element is non-zero otherwise it is zero
            if random.random() < density: 
                row.append(random.randint(elements_range[0], elements_range[1]))
            else:
                row.append(0)
        matrix.append(row)
    return matrix

In [4]:
sparse_matrix = generateRandomSparseMatrix((10, 10), 0.1, (-10, 10))

In [5]:
print("Sparse Matrix:")
for row in sparse_matrix:
    print(row)

Sparse Matrix:
[0, 0, 0, 0, -5, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 4, 0, 0, 0, 0, 0, -4]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, -8, 0, 0, 0, -7, -7, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[6, 9, 0, 0, 0, 0, 0, 0, 0, 0]


In [6]:
# Converting the Matrix to CSR Format
csrA = CSRMatrixRepresentation(sparse_matrix)

In [7]:
print("Matrix in CSR format:") # Print matrix in CSR format
print(csrA)

Matrix in CSR format:
data: [-5, 4, -4, -8, -7, -7, 6, 9]
cols: [4, 3, 9, 3, 7, 8, 0, 1]
rows: [0, 1, 1, 1, 3, 3, 3, 6, 6, 6, 8]


In [8]:
# Generating a Random Vector
vector = [random.randint(-10, 10) for i in range(10)]
print("Vector: ", vector)

Vector:  [1, 9, 1, -1, -7, -1, 0, 7, 3, -5]


In [9]:
print("Matrix Vector Multiplication Result: ", csrA.matrixVectorMultiply(vector)) # Print matrix-vector multiplication result

Matrix Vector Multiplication Result:  [35, 0, 0, 16, 0, 0, -62, 0, 0, 87]


In [10]:
# Computing using Numpy for comparision
import numpy as np 
matrix = np.array(sparse_matrix)
vector = np.array(vector)

print("Numpy Matrix Vector Multiplication Result: ", matrix.dot(vector))

Numpy Matrix Vector Multiplication Result:  [ 35   0   0  16   0   0 -62   0   0  87]


In [11]:
# Comapring time for a large sparse matrix
sparse_matrix = generateRandomSparseMatrix((100, 100), 0.01, (-10, 10))
csrA = CSRMatrixRepresentation(sparse_matrix)
vector = [random.randint(-10, 10) for i in range(100)]

In [12]:
start = time.time() # Start timer
csrA.matrixVectorMultiply(vector)
end = time.time() # End timer
print("Time taken for large sparse matrix: ", round(end - start, 10), " seconds") # Print time taken

matrix = np.array(sparse_matrix)
vector = np.array(vector)

start = time.time() # Start timer
matrix.dot(vector)
end = time.time() # End timer
print("Time taken for large sparse matrix using Numpy: ", round(end - start, 10), " seconds") # Print time taken

Time taken for large sparse matrix:  0.0001249313  seconds
Time taken for large sparse matrix using Numpy:  0.0003130436  seconds


The ```numpy``` library is written in C and is well optimised for matrix-vector operations. If we would have implemented the matrix-vector multiplication in Python using nested for loops and lists, it would have been slower than the ```numpy``` implementation. 

We can see that the sparse representation is running equally fast as the optimised ```numpy``` implementation. Hence, the CSR representation is a good way to represent sparse matrices.